In [1]:
!pip install datasets sentence-transformers faiss-cpu torch transformers pandas scikit-learn sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 36.6 MB/s eta 0:00:00


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from google.colab import files


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataset = load_dataset("KryptoniteCrown/synthetic-neurology-QA-dataset", split="train")
neurology_qa = [{"question": row["question"], "answer": row["answer"]} for row in dataset]

cardiology_dataset = load_dataset("ilyassacha/cardiology_qa", split="train")
cardiology_qa = [{"question": row["question"], "answer": row["answer"]} for row in cardiology_dataset]

dermatology_dataset = load_dataset("Mreeb/Dermatology-Question-Answer-Dataset-For-Fine-Tuning", split="train")
dermatology_qa = [{"question": row["prompt"], "answer": row["response"]} for row in dermatology_dataset]


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

reformatted_data_neurology.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1452 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/438 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/14885 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1654 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/929 [00:00<?, ?B/s]

combined_data.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1460 [00:00<?, ? examples/s]

In [50]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(filename)
display(df)

Disease_Control_and_PreventionQA_dataset = [{"question": row["Question"], "answer": row["Answer"]} for _, row in df.iterrows()]

Saving Disease_Control_and_PreventionQA.csv to Disease_Control_and_PreventionQA.csv


,Question,Answer,topic,split
0,What is (are) Acanthamoeba - Granulomatous Ame...,"Acanthamoeba is a microscopic, free-living ame...",Disease_Control_Prevention,train
1,Who is at risk for Acanthamoeba - Granulomatou...,Acanthamoeba keratitis is a rare disease that ...,Disease_Control_Prevention,train
2,How to diagnose Acanthamoeba - Granulomatous A...,Early diagnosis is essential for effective tre...,Disease_Control_Prevention,train
3,What are the treatments for Acanthamoeba - Gra...,Early diagnosis is essential for effective tre...,Disease_Control_Prevention,train
4,How to prevent Acanthamoeba - Granulomatous Am...,Topics,Disease_Control_Prevention,train
...,...,...,...,...
265,What is (are) Parasites - Ascariasis ?,Ascaris is an intestinal parasite of humans. I...,Disease_Control_Prevention,train
266,Who is at risk for Parasites - Ascariasis? ?,Ascaris infection is one of the most common in...,Disease_Control_Prevention,test
267,How to diagnose Parasites - Ascariasis ?,The standard method for diagnosing ascariasis ...,Disease_Control_Prevention,train
268,What are the treatments for Parasites - Ascari...,Anthelminthic medications (drugs that rid the ...,Disease_Control_Prevention,test


In [51]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(filename)
display(df)

cancer_dataset = [{"question": row["Question"], "answer": row["Answer"]} for _, row in df.iterrows()]

Saving CancerQA.csv to CancerQA.csv


,Question,Answer,topic,split
0,What is (are) Non-Small Cell Lung Cancer ?,Non-small cell lung cancer is a disease in whi...,cancer,train
1,Who is at risk for Non-Small Cell Lung Cancer? ?,Smoking is the major risk factor for non-small...,cancer,train
2,What are the symptoms of Non-Small Cell Lung C...,Signs of non-small cell lung cancer include a ...,cancer,test
3,How to diagnose Non-Small Cell Lung Cancer ?,Tests that examine the lungs are used to detec...,cancer,train
4,What is the outlook for Non-Small Cell Lung Ca...,Certain factors affect prognosis (chance of re...,cancer,train
...,...,...,...,...
724,How to diagnose Adult Soft Tissue Sarcoma ?,Adult soft tissue sarcoma is diagnosed with a ...,cancer,test
725,What is the outlook for Adult Soft Tissue Sarc...,Certain factors affect treatment options and p...,cancer,test
726,What are the stages of Adult Soft Tissue Sarco...,Key Points\n - After adult ...,cancer,train
727,what research (or clinical trials) is being do...,Patients may want to think about taking part i...,cancer,train


In [7]:
device = torch.device('cpu')  # keep CPU-safe for your environment
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device='cpu')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [52]:
def build_faiss_index(docs, domain_name="", max_docs=2000):
    if not docs:
        return None, []
    docs_subset = docs[:min(max_docs, len(docs))]
    texts = [d["question"] for d in docs_subset]
    embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=False, batch_size=32, device='cpu')
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings.astype(np.float32))
    return index, docs_subset

vector_dbs = {}
neurology_index, neurology_docs = build_faiss_index(neurology_qa, "Neurology")
cardio_index, cardio_docs = build_faiss_index(cardiology_qa, "Cardiology")
derma_index, derma_docs = build_faiss_index(dermatology_qa, "Dermatology")
cancer_index, cancer_docs = build_faiss_index(cancer_dataset, "Cancer")
disease_index, disease_docs = build_faiss_index(Disease_Control_and_PreventionQA_dataset, "Disease")

if neurology_index: vector_dbs["Neurology"] = (neurology_index, neurology_docs)
if cardio_index:   vector_dbs["Cardiology"] = (cardio_index, cardio_docs)
if derma_index:    vector_dbs["Dermatology"] = (derma_index, derma_docs)
if cancer_index:   vector_dbs["Cancer"] = (cancer_index, cancer_docs)
if disease_index:  vector_dbs["Disease"] = (disease_index, disease_docs)

print(f"Built indexes for domains: {list(vector_dbs.keys())}")

Built indexes for domains: ['Neurology', 'Cardiology', 'Dermatology', 'Cancer', 'Disease']


In [54]:
class MedicalExpert(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=512, output_dim=384):
        super().__init__()
        # self.network = nn.Sequential(
        #     nn.Linear(input_dim, hidden_dim),
        #     nn.ReLU(),
        #     nn.Dropout(0.1),
        #     nn.Linear(hidden_dim, output_dim)
        # )
        self.network = nn.Sequential(
          nn.Linear(input_dim, hidden_dim),
          nn.Tanh(),
          nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [55]:
class GatingNetwork(nn.Module):
    def __init__(self, input_dim=384, num_experts=3, hidden_dim=256):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_dim, num_experts)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.fc2(x)

In [56]:
# -------------------------
# Prepare supervised training data for router
# -------------------------
import random # Import random module
# Build mapping domain->label
domain_list = sorted(list(vector_dbs.keys()))
domain_to_label = {d: i for i, d in enumerate(domain_list)}
label_to_domain = {i: d for d, i in domain_to_label.items()}
num_classes = len(domain_list)
print("Domain->label:", domain_to_label)


# Create labeled training_data by sampling questions from each domain
training_data = []
samples_per_domain = 500  # adjust if your domains are small
for domain, (idx, docs) in vector_dbs.items():
    label = domain_to_label[domain]
    # shuffle docs and get up to samples_per_domain
    choices = docs.copy()
    random.shuffle(choices)
    for doc in choices[:samples_per_domain]:
        training_data.append({"question": doc["question"], "label": label})

if len(training_data) == 0:
    raise SystemExit("No training data constructed. Ensure your datasets exist.")

# Split
train_data, val_data = train_test_split(training_data, test_size=0.2, random_state=42, stratify=[d["label"] for d in training_data])

Domain->label: {'Cancer': 0, 'Cardiology': 1, 'Dermatology': 2, 'Disease': 3, 'Neurology': 4}


In [57]:
class MedicalMoE(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=512, output_dim=384, num_experts=3, top_k=2):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = min(top_k, num_experts)
        self.experts = nn.ModuleList([MedicalExpert(input_dim, hidden_dim, output_dim) for _ in range(num_experts)])
        self.gating = GatingNetwork(input_dim, num_experts)
        # expert_names can be set later
        self.expert_names = None

    def forward(self, x, return_router_logits=False):
        # x: (batch, input_dim)
        router_logits = self.gating(x)  # (batch, num_experts)
        if return_router_logits:
            return router_logits
        router_probs = F.softmax(router_logits, dim=-1)
        topk_probs, topk_indices = torch.topk(router_probs, self.top_k, dim=1)
        topk_probs = topk_probs / topk_probs.sum(dim=1, keepdim=True)
        # combine expert outputs (note: experts may be untrained)
        batch_size = x.size(0)
        out = torch.zeros(batch_size, x.size(1), device=x.device)
        for i in range(self.top_k):
            expert_idx_col = topk_indices[:, i]
            weights = topk_probs[:, i]
            # compute expert outputs for all batch where that expert is used
            for b in range(batch_size):
                eidx = expert_idx_col[b].item()
                out[b] += weights[b] * self.experts[eidx](x[b].unsqueeze(0)).squeeze(0)
        return out, topk_indices

    def get_expert_names(self, indices):
        # indices: tensor shape (batch, top_k)
        if self.expert_names is None:
            return [f"expert_{i}" for i in indices[0].tolist()]
        return [self.expert_names[idx.item()] for idx in indices[0]]


In [58]:
# Dataset class
class MedicalQueryDataset(Dataset):
    def __init__(self, data, embedder):
        self.data = data
        self.questions = [d["question"] for d in data]
        self.labels = [d["label"] for d in data]
        # encode once
        self.embeddings = embedder.encode(self.questions, convert_to_numpy=True, show_progress_bar=False, batch_size=32, device='cpu').astype(np.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return {"embedding": torch.from_numpy(self.embeddings[idx]), "label": torch.tensor(self.labels[idx], dtype=torch.long)}

train_dataset = MedicalQueryDataset(train_data, embedder)
val_dataset   = MedicalQueryDataset(val_data, embedder)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

Train size: 1816, Val size: 454


In [59]:
# -------------------------
# Train router (gating network) only — conservative approach
# -------------------------
moe_model = MedicalMoE(input_dim=train_dataset.embeddings.shape[1], num_experts=num_classes, top_k=2)
moe_model.expert_names = domain_list  # map indices to domain names

def train_router(model, train_loader, val_loader, epochs=5, lr=1e-3):
    model.to(device)
    optimizer = torch.optim.Adam(model.gating.parameters(), lr=lr)  # only train gating
    criterion = nn.CrossEntropyLoss()
    best_val = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for batch in train_loader:
            emb = batch["embedding"].to(device)
            labels = batch["label"].to(device)
            optimizer.zero_grad()
            logits = model(emb, return_router_logits=True)  # (batch, num_experts)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * emb.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        train_acc = 100.0 * correct / total
        avg_loss = running_loss / total

        # val
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for batch in val_loader:
                emb = batch["embedding"].to(device)
                labels = batch["label"].to(device)
                logits = model(emb, return_router_logits=True)
                preds = logits.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)
        val_acc = 100.0 * val_correct / val_total
        print(f"Epoch {epoch+1}/{epochs} Loss={avg_loss:.4f} TrainAcc={train_acc:.2f}% ValAcc={val_acc:.2f}%")
        if val_acc > best_val:
            best_val = val_acc
            # save gating only (lighter)
            torch.save({"gating_state_dict": model.gating.state_dict(), "num_experts": num_classes}, "router_gating.pt")
            print("  Saved router_gating.pt")
    return model

print("Training router (this may take a few minutes)...")
moe_model = train_router(moe_model, train_loader, val_loader, epochs=5, lr=1e-3)

# keep for inference
trained_moe_model = moe_model
CHECKPOINT_PATH = "best_moe_model.pt"
torch.save({
    'model_state_dict': trained_moe_model.state_dict(),
    'num_experts': num_classes
}, CHECKPOINT_PATH)
print("Saved full model checkpoint (router + experts) to:", CHECKPOINT_PATH)

Training router (this may take a few minutes)...
Epoch 1/5 Loss=1.1792 TrainAcc=75.06% ValAcc=85.90%
  Saved router_gating.pt
Epoch 2/5 Loss=0.3766 TrainAcc=92.46% ValAcc=94.49%
  Saved router_gating.pt
Epoch 3/5 Loss=0.1984 TrainAcc=95.10% ValAcc=95.81%
  Saved router_gating.pt
Epoch 4/5 Loss=0.1418 TrainAcc=96.48% ValAcc=95.37%
Epoch 5/5 Loss=0.1051 TrainAcc=97.80% ValAcc=96.04%
  Saved router_gating.pt
Saved full model checkpoint (router + experts) to: best_moe_model.pt


In [60]:
try:
    hyde_tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT")
    hyde_model = AutoModelForCausalLM.from_pretrained("microsoft/BioGPT")
    hyde_model.eval()
    print("✓ HyDE (BioGPT) loaded")
except:
    hyde_model = None
    print("⚠ HyDE not available")

try:
    reranker_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
    reranker_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
    reranker_model.eval()
    print("✓ Reranker (FLAN-T5) loaded")
except:
    reranker_model = None
    print("⚠ Reranker not available")

✓ HyDE (BioGPT) loaded
✓ Reranker (FLAN-T5) loaded


In [64]:
def hyde_hypothesis(query, max_new_tokens=100):
    """
    Generate a clean hypothetical medical answer (HyDE step).
    Ensures the generated text directly addresses the main focus of the question.
    """
    if hyde_model is None:
        return query
    try:
        prompt = f"""
You are a highly knowledgeable medical expert.
Given the following question, generate a concise, evidence-based hypothetical answer that could appear in a clinical textbook.
Focus on:
- Listing key symptoms, diagnostic clues, and treatments if applicable.
- Using factual reasoning, not imagination.
- Staying strictly within the topic domain of the question.

Question: {query}
Answer:
"""

        print("\n[HyDE] Generating hypothetical medical answer...")
        inputs = hyde_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256)
        outputs = hyde_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.4,
            do_sample=False,
            pad_token_id=hyde_tokenizer.eos_token_id
        )
        return hyde_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    except Exception as e:
        print(f"[HyDE] Error: {e}")
        return query


In [62]:
def llm_rerank(query, candidate_answers):
    """Rerank with keyword fallback"""
    if not candidate_answers:
        return []

    if reranker_model is None or reranker_tokenizer is None:
        return keyword_based_rerank(query, candidate_answers)

    scored_answers = []

    for ans in candidate_answers[:5]:
        try:
            prompt = f"Rate from 0 to 10 how well the following answer addresses the question.\n\nQuestion: {query}\nAnswer: {ans}\nScore:"
            inputs = reranker_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=400)

            with torch.no_grad():
                outputs = reranker_model.generate(**inputs, max_new_tokens=3, num_beams=1, do_sample=False)

            score_text = reranker_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

            import re
            numbers = re.findall(r'\d+\.?\d*', score_text)

            if numbers:
                score = min(1.0, float(numbers[0]) / 10.0)
            else:
                score = keyword_score(query, ans)
        except:
            score = keyword_score(query, ans)

        scored_answers.append({"answer": ans, "reranker_score": score})

    return sorted(scored_answers, key=lambda x: x["reranker_score"], reverse=True)


def keyword_score(query, answer):
    """Keyword scoring"""
    query_words = set(query.lower().split())
    answer_words = set(answer.lower().split())

    if len(query_words) == 0:
        return 0.3

    overlap = len(query_words & answer_words)
    score = overlap / len(query_words)

    medical_terms = ['treatment', 'symptoms', 'diagnosis', 'disease', 'therapy', 'medication', 'condition']
    for term in medical_terms:
        if term in query.lower() and term in answer.lower():
            score += 0.1

    return min(1.0, score)


def keyword_based_rerank(query, candidates):
    """Pure keyword reranking"""
    scored = []
    for ans in candidates[:5]:
        scored.append({"answer": ans, "reranker_score": keyword_score(query, ans)})
    return sorted(scored, key=lambda x: x["reranker_score"], reverse=True)



In [ ]:
def llm_rerank(query, candidate_answers, candidate_similarities=None):
    """
    Rerank answers using LLM-based scoring + embedding similarity blending.
    - query: user query text
    - candidate_answers: list of retrieved answers
    - candidate_similarities: optional list of FAISS similarity scores (normalized)
    """
    if not candidate_answers:
        return []

    # Normalize embedding similarities if available
    if candidate_similarities is not None and len(candidate_similarities) > 0:
        import numpy as np
        # Normalize to 0–1 range
        min_s, max_s = min(candidate_similarities), max(candidate_similarities)
        if max_s > min_s:
            candidate_similarities = [
                (s - min_s) / (max_s - min_s) for s in candidate_similarities
            ]
        else:
            candidate_similarities = [0.5] * len(candidate_answers)
    else:
        candidate_similarities = [0.5] * len(candidate_answers)

    scored_answers = []

    for i, ans in enumerate(candidate_answers[:5]):
        # Default reranker fallback
        rerank_score = 0.0
        try:
            prompt = (
                f"Rate from 0 to 10 how well the following answer addresses the question.\n\n"
                f"Question: {query}\nAnswer: {ans}\nScore:"
            )
            inputs = reranker_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=400)

            with torch.no_grad():
                outputs = reranker_model.generate(
                    **inputs, max_new_tokens=3, num_beams=1, do_sample=False
                )

            score_text = reranker_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
            import re
            numbers = re.findall(r'\d+\.?\d*', score_text)
            if numbers:
                rerank_score = min(1.0, float(numbers[0]) / 10.0)
            else:
                rerank_score = keyword_score(query, ans)
        except Exception:
            rerank_score = keyword_score(query, ans)

        # Combine embedding + reranker scores
        embedding_score = candidate_similarities[i]
        final_score = 0.7 * embedding_score + 0.3 * rerank_score

        scored_answers.append({
            "answer": ans,
            "embedding_score": embedding_score,
            "reranker_score": rerank_score,
            "final_score": final_score,
        })

    return sorted(scored_answers, key=lambda x: x["final_score"], reverse=True)


In [63]:
# -------------------------
# Retrieval function (Final Version): integrates HyDE + LLM Reranker
# -------------------------
def retrieve_answer_with_moe_fixed(query, k=5, use_hyde=True):
    # 1️⃣ HyDE generation (optional)
    if use_hyde and hyde_model is not None:
        hyde_text = hyde_hypothesis(query)
        orig_emb = embedder.encode([query], convert_to_numpy=True).astype(np.float32)
        hyde_emb = embedder.encode([hyde_text], convert_to_numpy=True).astype(np.float32)
        query_emb = np.mean([orig_emb, hyde_emb], axis=0)
    else:
        query_emb = embedder.encode([query], convert_to_numpy=True).astype(np.float32)

    # 3️⃣ Route query through Mixture of Experts (MoE) model
    with torch.no_grad():
        q_tensor = torch.from_numpy(query_emb).to(device)
        logits = trained_moe_model(q_tensor, return_router_logits=True)  # (1, num_experts)
        probs = F.softmax(logits, dim=-1).cpu().numpy().squeeze(0)

        # Choose top-k domains
        topk = min(2, len(domain_list))
        top_indices = probs.argsort()[::-1][:topk]
        selected_domains = [label_to_domain[int(i)] for i in top_indices]
        selected_probs = [float(probs[int(i)]) for i in top_indices]

    # 4️⃣ Retrieve candidate documents from FAISS for the selected domains
    candidates = []
    for domain in selected_domains:
        if domain not in vector_dbs:
            continue
        idx, docs = vector_dbs[domain]
        D, I = idx.search(query_emb, k)
        for dist, doc_idx in zip(D[0], I[0]):
            if doc_idx < len(docs):
                candidates.append({
                    "answer": docs[doc_idx]["answer"],
                    "domain": domain,
                    "dist": float(dist)
                })

    if not candidates:
        return {
            "query": query,
            "selected_experts": selected_domains,
            "best_answer": "⚠️ No info found.",
            "confidence_score": 0.0
        }

    # 5️⃣ Extract candidate answers for reranking
    candidate_texts = [c["answer"] for c in candidates]

    # 6️⃣ Apply LLM-based reranker (FLAN-T5 or keyword fallback)
    reranked = llm_rerank(query, candidate_texts)

    if not reranked:
        best_answer = candidates[0]["answer"]
        conf = 1.0 / (1.0 + candidates[0]["dist"])
    else:
        best_answer = reranked[0]["answer"]
        conf = reranked[0]["reranker_score"]

    return {
        "query": query,
        "selected_experts": selected_domains,
        "best_answer": best_answer,
        "confidence_score": conf,
        "candidates_count": len(candidates)
    }

# -------------------------
# Quick test
# -------------------------
tests = [
    "What are common skin conditions and treatments?",
    "What are cardiovascular symptoms?",
    "What are neurological disorders?",
    "What public health measures control the spread of COVID-19?",
    "What are common types of cancer and their symptoms?",
    "What are the symptoms of Parkinson’s disease?"
]

for t in tests:
    r = retrieve_answer_with_moe_fixed(t, k=7, use_hyde=True)
    print("\n---")
    print("Query:", t)
    print("Selected domains:", r["selected_experts"])
    print("Confidence:", round(r["confidence_score"], 3))
    print("Answer:", r["best_answer"].strip().replace("\n", " "))




[HyDE] Generating hypothetical medical answer...

---
Query: What are common skin conditions and treatments?
Selected domains: ['Dermatology', 'Disease']
Confidence: 0.4
Answer: Dermatitis is a general term for inflammation of the skin. The types of dermatitis include Atopic dermatitis (eczema), Contact dermatitis, Seborrheic dermatitis, and Stasis dermatitis.   Symptoms of dermatitis can vary depending on the type but typically include redness, itching, swelling, and sometimes blistering, cracking, or scaling.   Atopic dermatitis is usually treated with medications like hydrocortisone creams to address the itching, as well as moisturizers for the dryness. Contact dermatitis treatment typically involves identifying and avoiding the allergens or irritants causing your symptoms. For seborrheic dermatitis, a person might use a dandruff shampoo and use a mild steroid lotion. Treatment of stasis dermatitis may include compresses, wet dressings, and topical corticosteroids.

[HyDE] Generati